<a href="https://colab.research.google.com/github/kunishou/Talking_Robot/blob/main/01_whisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

# Whisper set up

In [ ]:
!pip install -q git+https://github.com/openai/whisper.git

In [ ]:
import whisper
model = whisper.load_model("small")

100%|██████████████████████████████████████| 2.87G/2.87G [00:15<00:00, 201MiB/s]


In [ ]:
# ローカルPCマイクでの録音設定

from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

In [ ]:
#@title Whisper Test

audiofile = "audio.wav"
second = 10 #@param {type:"number"}
print(f"Speak to your microphone {second} sec...")
record(second, audiofile)
print("Done!")

result = model.transcribe("audio.wav", verbose=False, language="ja")
print(result["text"])

In [ ]:
# GPUの停止
from google.colab import runtime
runtime.unassign()